In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: C:\Users\dania\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('police_fatality_without_header.csv', header=0, encoding='ISO-8859-1')


In [ ]:
# clean empty rows
df_cleaned = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
header = ['Person', 'Action', 'Date', 'City', 'Gender', 'Age']
df_cleaned.columns = header
df_cleaned.to_csv('police_fatality_with_header.csv', index=False)

Numpy

In [ ]:
pip install numpy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('police_fatality_with_header.csv')

In [ ]:
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, np.inf)]
age_labels = ['0-18', '18-30', '30-50', '50-70', '70+']

In [ ]:
df['Age Group'] = pd.cut(df['Age'], bins=[0, 18, 30, 50, 70, np.inf], labels=age_labels, right=False)

In [ ]:
percentage_by_state_and_age = df.groupby(['City', 'Age Group']).size().groupby(level=0).apply(
    lambda x: 100 * x / float(x.sum())
).unstack().fillna(0)

In [ ]:
#save output as csv file
percentage_by_state_and_age.to_csv('percentage_by_state_and_age.csv')

In [13]:
header = ['Person', 'Action', 'Date', 'City', 'Gender', 'Age']
df_cleaned.columns = header
df_cleaned.to_csv('police_fatality_with_header.csv', index=False)

Numpy

In [14]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: C:\Users\dania\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('police_fatality_with_header.csv')

In [5]:
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, np.inf)]
age_labels = ['0-18', '18-30', '30-50', '50-70', '70+']

In [6]:
df['Age Group'] = pd.cut(df['Age'], bins=[0, 18, 30, 50, 70, np.inf], labels=age_labels, right=False)

In [7]:
percentage_by_state_and_age = df.groupby(['City', 'Age Group']).size().groupby(level=0).apply(
    lambda x: 100 * x / float(x.sum())
).unstack().fillna(0)

C:\Users\dania\AppData\Local\Temp\ipykernel_12676\239723569.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_state_and_age = df.groupby(['City', 'Age Group']).size().groupby(level=0).apply(


In [8]:
#save output as csv file
percentage_by_state_and_age.to_csv('percentage_by_state_and_age.csv')

SQL

In [2]:
!pip install  sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
You should consider upgrading via the 'C:\Users\dania\Desktop\BigData\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import sqlite3

In [5]:
con = sqlite3.connect("end_term.db")

In [6]:
cur = con.cursor()

In [7]:
cur.execute("CREATE TABLE bigdata(Person,Action,Date,City,Gender,Age)")

In [9]:
import csv


In [12]:
with open('police_fatality_with_header.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    # Skip the header row if it exists
    next(csv_reader, None)

    for row in csv_reader:
        cur.execute("INSERT INTO bigdata VALUES (?, ?, ?, ?, ?, ?)", row)

con.commit()

Find the most shootings city

In [23]:
cur.execute("SELECT City, COUNT(*) as CrimeCount FROM bigdata WHERE Action = 'Shooting' GROUP BY City ORDER BY CrimeCount DESC LIMIT 1")
result = cur.fetchone()

if result:
    city, crime_count = result
    print(f"The city with the most shootings is {city} with {crime_count} occurrences.")
else:
    print("No data found for shootings in the dataset.")


The city with the most shootings is California with 507 occurrences.


The gander that breaks the lawe more

In [24]:
cur.execute("SELECT Gender, COUNT(*) as GenderCount FROM bigdata GROUP BY Gender ORDER BY GenderCount DESC LIMIT 1")
result = cur.fetchone()

if result:
    gender, count = result
    print(f"The gender with the most occurrences of breaking laws is {gender} with {count} occurrences.")
else:
    print("No gender data found in the dataset.")

The gender with the most occurrences of breaking laws is Male with 3882 occurrences.


The most frequent age of law breakers

In [27]:
cur.execute("SELECT Age, COUNT(*) as AgeCount FROM bigdata GROUP BY Age ORDER BY AgeCount DESC LIMIT 1")

# Fetch the result
result = cur.fetchone()

# Display the result
if result:
    age, count = result
    print(f"The most frequent age among law breakers is {age} with {count} occurrences.")
else:
    print("No age data found in the dataset.")
con.close()

The most frequent age among law breakers is 26 with 186 occurrences.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import col, count, desc, asc, rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, asc, rank
schema = StructType([
    StructField("Person", StringType(), True),
    StructField("Action", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Gender", StringType(), True),
    StructField("Age", IntegerType(), True),
])

spark = SparkSession.builder.appName("EndtermBDSpark").getOrCreate()
path = "police_fatality_with_header.csv"
df = spark.read.csv(path, header=True, inferSchema=True)
#to count deaths in each state
df = spark.read.csv(path, header=True, schema=schema)

action_counts = df.groupBy("City", "Action").count()

windowSpecDesc = Window.partitionBy("City").orderBy(desc("count"))
windowSpecAsc = Window.partitionBy("City").orderBy(asc("count"))
#making top most and less popular deaths

ranked_actions_desc = action_counts.withColumn("rank_desc", rank().over(windowSpecDesc))
ranked_actions_asc = action_counts.withColumn("rank_asc", rank().over(windowSpecAsc))

#most popular death reason
most_popular_actions = ranked_actions_desc.filter(col("rank_desc") == 1).select("City", "Action", "count").orderBy("City")
most_popular_actions = ranked_actions_desc.filter(col("rank_desc") == 1) \
    .select("City", "Action", "count") \
    .orderBy(desc("count"))
#least popular death reason
least_popular_actions = ranked_actions_asc.filter(col("rank_asc") == 1).select("City", "Action", "count").orderBy("City")
least_popular_actions = ranked_actions_asc.filter(col("rank_asc") == 1) \
    .select("City", "Action", "count") \
    .orderBy(asc("count"))

print("Most Popular Actions by City:")
print("Most Popular Actions by City (sorted by count descending):")
most_popular_actions.show(truncate=False)
print("Least Popular Actions by City:")
print("Least Popular Actions by City (sorted by count ascending):")
least_popular_actions.show(truncate=False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 18:38:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Most Popular Actions by City:
Most Popular Actions by City (sorted by count descending):


+--------------+--------+-----+
|City          |Action  |count|
+--------------+--------+-----+
|California    |Shooting|169  |
|Texas         |Shooting|122  |
|Florida       |Shooting|105  |
|Arizona       |Shooting|50   |
|Ohio          |Shooting|40   |
|North Carolina|Shooting|39   |
|Oklahoma      |Shooting|36   |
|Pennsylvania  |Shooting|34   |
|Georgia       |Shooting|33   |
|Colorado      |Shooting|31   |
|New York      |Shooting|29   |
|New Mexico    |Shooting|28   |
|Tennessee     |Shooting|27   |
|Illinois      |Shooting|27   |
|Missouri      |Shooting|26   |
|Washington    |Shooting|26   |
|Maryland      |Shooting|22   |
|Alabama       |Shooting|22   |
|Wisconsin     |Shooting|21   |
|Indiana       |Shooting|20   |
+--------------+--------+-----+
only showing top 20 rows

Least Popular Actions by City:
Least Popular Actions by City (sorted by count ascending):
+-----------+----------------------+-----+
|City       |Action                |count|
+-----------+-----------------